# Chain-of-Note RAG Tutorial

This notebook provides a step-by-step tutorial on how to use the Chain-of-Note RAG system to reduce hallucinations in AI-generated responses.

## Setup

First, let's set up our environment and import the necessary modules:

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from src.data_loader import DocumentLoader
from src.embeddings import SentenceTransformerEmbeddings
from src.document_store import DocumentStore
from src.chain_of_note import ChainOfNote
from src.rag_system import ChainOfNoteRAG
from src.evaluation import RAGEvaluator

print("Modules imported successfully")

## 1. Create Sample Documents

Let's create some sample documents for our RAG system:

In [ ]:
documents = [
    {
        "content": "Chain-of-Note is a technique for improving RAG systems by generating intermediate notes. By first creating structured notes from retrieved documents, the model can better organize information before generating a response. This reduces hallucinations and improves factual accuracy.",
        "metadata": {"source": "Research Paper", "topic": "Chain-of-Note"}
    },
    {
        "content": "Hallucinations in large language models occur when the model generates information that seems plausible but is factually incorrect or not grounded in the provided context. This is a common issue in generative AI that researchers are actively working to address.",
        "metadata": {"source": "AI Blog", "topic": "Hallucinations"}
    },
    {
        "content": "Retrieval-Augmented Generation (RAG) combines retrieval mechanisms with text generation to produce outputs that are grounded in external knowledge. By retrieving relevant information before generation, RAG can improve factuality compared to standard language models.",
        "metadata": {"source": "Academic Paper", "topic": "RAG"}
    }
]

print(f"Created {len(documents)} sample documents")

## 2. Initialize the RAG System

Now, let's initialize our Chain-of-Note RAG system:

In [ ]:
# Initialize the embedding model
print("Initializing embedding model...")
embedding_model = SentenceTransformerEmbeddings()

# Initialize the RAG system
print("Initializing Chain-of-Note RAG system...")
rag_system = ChainOfNoteRAG(embedding_model=embedding_model)

print("System initialized successfully")

## 3. Add Documents to the RAG System

Let's add our sample documents to the RAG system:

In [ ]:
# Add documents to the RAG system
rag_system.add_documents(documents)
print(f"Added {len(documents)} documents to the RAG system")

## 4. Query the RAG System

Now let's query our system with a question:

In [ ]:
# Define a query
query = "How does Chain-of-Note help reduce hallucinations in RAG systems?"

# Query the system
response = rag_system.query(
    query=query,
    top_k=3,
    return_context=True,
    return_notes=True
)

# Display the notes
print("Generated Notes:")
print("-" * 60)
print(response["notes"])
print("-" * 60)

# Display the answer
print("\nFinal Answer:")
print("-" * 60)
print(response["answer"])
print("-" * 60)

## 5. Examine Retrieved Documents

Let's see which documents were retrieved:

In [ ]:
print("Retrieved Documents:")
for i, doc in enumerate(response["context"], 1):
    print(f"Document {i} (Score: {doc['score']:.4f}):")
    print(f"Topic: {doc['metadata'].get('topic', 'Unknown')}")
    print(f"Source: {doc['metadata'].get('source', 'Unknown')}")
    print(f"Content: {doc['content']}")
    print()

## 6. Compare with Standard RAG

Now let's implement a basic RAG approach without the Chain-of-Note technique for comparison:

In [ ]:
# Basic RAG implementation without notes
def basic_rag(query, documents, model):
    prompt = f"Question: {query}\n\nContext Information:\n"
    
    for i, doc in enumerate(documents, 1):
        content = doc["content"]
        prompt += f"[Document {i}]: {content}\n\n"
        
    prompt += "Based on the above context information, please answer the question concisely.\n\nAnswer:"
    
    # Generate answer directly
    result = model.generator(prompt, max_length=512, num_return_sequences=1)
    answer = result[0]["generated_text"]
    
    return answer.strip()

# Get standard RAG answer
standard_answer = basic_rag(query, response["context"], rag_system.chain_of_note)

print("Standard RAG Answer:")
print("-" * 60)
print(standard_answer)
print("-" * 60)

print("\nChain-of-Note RAG Answer:")
print("-" * 60)
print(response["answer"])
print("-" * 60)

## 7. Evaluate the Responses

Let's evaluate both approaches to see which one performs better:

In [ ]:
# Initialize evaluator
evaluator = RAGEvaluator()

# Prepare reference docs for evaluation
reference_docs = [doc["content"] for doc in response["context"]]

# Compare systems
comparison = evaluator.compare_systems(
    query=query,
    standard_rag_response=standard_answer,
    chain_of_note_response=response["answer"],
    reference_docs=reference_docs
)

import pandas as pd
import matplotlib.pyplot as plt

# Create a comparison table
metrics = {
    "Hallucination Score": [
        comparison["standard_rag"]["hallucination_score"],
        comparison["chain_of_note_rag"]["hallucination_score"]
    ]
}

if "relevance" in comparison["standard_rag"]:
    metrics["Relevance"] = [
        comparison["standard_rag"]["relevance"],
        comparison["chain_of_note_rag"]["relevance"]
    ]

df = pd.DataFrame(metrics, index=["Standard RAG", "Chain-of-Note RAG"])
df

In [ ]:
# Visualize the comparison
ax = df.plot(kind="bar", figsize=(10, 6))
plt.title("Comparison: Standard RAG vs Chain-of-Note RAG")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.legend(loc="best")

# Add value labels
for container in ax.containers:
    ax.bar_label(container, fmt='%.3f')

plt.tight_layout()
plt.show()

## 8. Conclusion

As shown in the evaluation, Chain-of-Note RAG typically performs better than standard RAG in terms of hallucination reduction. The intermediate note-taking step helps the model organize information from the retrieved documents before generating a response, leading to more accurate and factual answers.